In [20]:
import numpy as np
import gtda
import matplotlib.pyplot as plt
from pathlib import Path #auxilliary module




In [27]:
import os

import sys

# Add the directory containing the 'data' folder to the Python path
sys.path.append(r'C:\Users\Joe\Documents\University\4th Year\symmetric_tda\gtda')


There are 30 point clouds in 3 dimensions, each with 100 points.


In [3]:
class Space:
    """
    Create a space object with a name and an embedding dimension
    """
    def __init__(self, name, edim):
        self.name = name
        self.edim = edim

    def sample(self, n):
        raise NotImplementedError()

    def poincare_polynomial(self):
        raise NotImplementedError()

    def barcode(self):
        raise NotImplementedError()

In [4]:
class Sphere(Space):
    def __init__(self, dim):
        """
        Create a sphere of dimension dim.  This is the set of unit vectors 
        in R ^(dim + 1), so the embedding dimension is dim+1.
        """
        self.dim = dim
        super().__init__('S^{' + str(dim) + '}', dim+1)

    def sample(self, n):
        """
        Returns a numpy matrix of shape n x edim, where each row is a
        random point on the sphere.  To do this, we generate a matrix
        of shape n x edim of standard normal random variables, and then
        normalize each row to have unit length.
        """
        x = np.random.randn(n, self.edim)
        x /= np.linalg.norm(x, axis=1)[:, None]
        return x

    def poincare_polynomial(self):
        """
        This returns the correct answer for the Poincare polynomial 
        of the sphere, represented as an integer-valued numpy array.
        The i-th entry is the coefficient of t^i, which is the rank
        of the i-th homology group.  In the case of the sphere, the
        Poincare polynomial is 1 + t^dim.
        """
        p = np.zeros(self.dim+1, dtype=int)
        p[0] = 1
        p[self.dim] = 1
        return p


In [33]:
class ComplexProjectiveSpace(Space):
    def __init__(self, cdim):
        self.cdim = cdim
        super().__init__('CP^{' + str(cdim) + '}', (cdim+1) ** 2)

    def sample(self, n):
        d = self.cdim+1
        x = np.random.randn(n, d) + 1j * np.random.randn(n, d)
        x /= np.linalg.norm(x, axis=1)[:, None]
        return x.reshape(n,d,1) * np.conj(x.reshape(n, 1, d))

    def poincare_polynomial(self):
        d = self.cdim
        p = np.zeros(2*d + 2, dtype=int)
        p[0::2] = 1
        return p

In [6]:
X = Sphere(4)
print(X.name)
print(X.dim)
print(X.edim)

S^{4}
4
5


In [7]:
X = ComplexProjectiveSpace(3)
X.name

'CP^{3}'

In [8]:
X.cdim

3

In [34]:
x = X.sample(3)

In [35]:
x

array([[[ 0.15854341+1.52129106e-18j,  0.0708904 +8.74111933e-02j,
         -0.1813381 -1.01109377e-01j, -0.27813283+1.66357520e-02j],
        [ 0.0708904 -8.74111933e-02j,  0.07989084-6.03315543e-19j,
         -0.13682828+5.47691945e-02j, -0.11519115+1.60783963e-01j],
        [-0.1813381 +1.01109377e-01j, -0.13682828-5.47691945e-02j,
          0.27189154-4.25408998e-18j,  0.30751229-1.96403828e-01j],
        [-0.27813283-1.66357520e-02j, -0.11519115-1.60783963e-01j,
          0.30751229+1.96403828e-01j,  0.48967422-2.27655266e-18j]],

       [[ 0.25830572-6.77055663e-18j, -0.27718038-6.92732762e-02j,
          0.11608844+2.80074695e-02j, -0.0585632 -3.03752415e-01j],
        [-0.27718038+6.92732762e-02j,  0.31601217+5.72220547e-18j,
         -0.13208228+1.07897521e-03j,  0.1443038 +3.10242246e-01j],
        [ 0.11608844-2.80074695e-02j, -0.13208228-1.07897521e-03j,
          0.05520956-1.38068844e-18j, -0.05925478-1.30163348e-01j],
        [-0.0585632 +3.03752415e-01j,  0.1443038 -3.1

In [11]:
np.linalg.norm(x[0] @ x[0] - x[0]) 

3.6399681661659405e-16

In [36]:
d = 2
n = 1
x = np.random.randn(n, d) + 1j * np.random.randn(n, d)
x /= np.linalg.norm(x, axis=1)[:, None]
y = x.reshape(n,d,1) * np.conj(x.reshape(n, 1, d))


In [13]:
x

array([[ 0.22968231-0.17672857j, -0.85469184-0.43071442j]])

In [37]:
x1 = np.conj(x.reshape(n, 1, d))
x1

array([[[0.28094228+0.25047815j, 0.91368863+0.15331409j]]])

In [38]:
x1 * x.reshape(n,d,1)

array([[[0.14166787+5.50572026e-18j, 0.2950956 -1.85786628e-01j],
        [0.2950956 +1.85786628e-01j, 0.85833213+3.12927904e-18j]]])

In [16]:
a = np.array([[1,2],[3,4],[5,6]])
print(a.shape)
print(a)


(3, 2)
[[1 2]
 [3 4]
 [5 6]]


In [17]:
a.reshape(3,1,2)

array([[[1, 2]],

       [[3, 4]],

       [[5, 6]]])

In [45]:
from gtda.generate_datasets import make_point_clouds
n_samples_per_class = 12
point_clouds, labels = make_point_clouds(n_samples_per_class, 10, 0.1)
point_clouds.shape
print(f"There are {point_clouds.shape[0]} point clouds in {point_clouds.shape[2]} dimensions, "
      f"each with {point_clouds.shape[1]} points.")

There are 36 point clouds in 3 dimensions, each with 100 points.


In [56]:
from gtda.homology import VietorisRipsPersistence

VR = VietorisRipsPersistence(homology_dimensions=[0,1,2])  # Parameter explained in the text
diagrams = VR.fit_transform(point_clouds)
diagrams.shape

(36, 171, 3)

In [181]:
sphere = Sphere(dim=2)  # This has been defined in the prev

# Generate 4 different samples of 70 points each on the sphere
samples = [sphere.sample(300) for _ in range(4)]

# Reshape each sample to add a batch dimension
samples = [points.reshape(1, *points.shape) for points in samples]

# Combine all samples into a single array with a batch dimension
points = np.concatenate(samples, axis=0)

In [182]:
from gtda.homology import VietorisRipsPersistence

VR = VietorisRipsPersistence(homology_dimensions=[0,1,2])  # Parameter explained in the text
diagrams = VR.fit_transform(points)
diagrams.shape

(4, 388, 3)

In [1]:
from gtda.plotting import plot_diagram

i = 3
fig, ax = plt.subplots(3, 1)
for i in range(3):
    plot_diagram(diagrams[i], ax=ax[i])


NameError: name 'plt' is not defined